This notebook formats election results at the regional level. It can be adapted later for another level of aggregation.

In [30]:
from typing import Dict, Set

import numpy as np
import pandas as pd

AFFILIATIONS = {
    "farleft": [
        "LFG",
        "LXG",
        "LPC",
        "FG",
        "BC-FG",
        "LEXG",
        "LCOP",
        "FI",
        "EXG",
        "COM",
        "LAGUILLER",
        "BESANCENOT",
        "BUFFET",
        "MÉLENCHON",
        "LA FRANCE INSOUMISE",
    ],
    "left": [
        "LPS",
        "LUG",
        "LSOC",
        "HAMON",
        "RDG",
        "SOC",
        "BC-SOC",
        "BC-UG",
        "JOSPIN",
        "ROYAL",
        "HOLLANDE",
        "ENVIE D'EUROPE",
        "LGA",
    ],
    "green": [
        "LVEC",
        "LVE",
        "LEC",
        "VEC",
        "BC-VEC",
        "ECO",
        "MAMERE",
        "BOVÉ",
        "VOYNET",
        "JOLY",
        "EUROPE ÉCOLOGIE",
    ],
    "center": [
        "LCMD",
        "MACRON",
        "MDM",
        "BC-MDM",
        "BC-UC",
        "REM",
        "CEN",
        "NCE",
        "LUC",
        "UDFD",
        "BAYROU",
        "UDF",
        "LUDF",
        "M-NC",
        "MODM",
        "RENAISSANCE",
        "LDR",
    ],
    "right": [
        "LUMP",
        "LUDI",
        "BC-UDI" "LUD",
        "LMAJ",
        "LR",
        "UDI",
        "UMP",
        "BC-UMP",
        "BC-UD",
        "MAJ",
        "CHIRAC",
        "SARKOZY",
        "FILLON",
        "UNION DROITE-CENTRE",
        "LDR",
    ],
    "farright": ["LFN", "LE PEN", "FN", "BC-FN", "PRENEZ LE POUVOIR"],
}
# not using department elections 2015, at least for now, because alliances all over the place
ELECTIONS = {
    "dep2004": {
        "date": "2004-03-21",
        "file": "dep2004.xls",
        "sheet": "Régions T1",
        "denom": "Code Nuance",
        "type": "department",
    },
    "dep2008": {
        "date": "2008-03-09",
        "file": "dep2008.xls",
        "sheet": "Régions T1",
        "denom": "Code Nuance",
        "type": "department",
    },
    "dep2011": {
        "date": "2011-03-20",
        "file": "dep2011.xls",
        "sheet": "Régions T1",
        "denom": "Code Nuance",
        "type": "department",
    },
    "euro2004": {
        "date": "2004-06-13",
        "file": "euro2004.xls",
        "sheet": "Regions",
        "denom": "Nuance Liste",
        "type": "european",
    },
    "euro2009": {
        "date": "2009-06-07",
        "file": "euro2009.xls",
        "sheet": "Regions",
        "denom": "Nuance Liste",
        "type": "european",
    },
    "euro2014": {
        "date": "2014-05-25",
        "file": "euro2014.xlsx",
        "sheet": "Régions",
        "denom": "Nuance Liste",
        "type": "european",
    },
    "euro2019": {
        "date": "2019-05-26",
        "file": "euro2019-regions.xls",
        "sheet": "Regions",
        "denom": "Libellé Abrégé Liste",
        "type": "european",
    },
    "legis2002": {
        "date": "2002-06-09",
        "file": "leg2002.xls",
        "sheet": "Régions T1",
        "denom": "Code Nuance",
        "type": "legislative",
    },
    "legis2007": {
        "date": "2007-06-10",
        "file": "leg2007.xls",
        "sheet": "Régions T1",
        "denom": "Code Nuance",
        "type": "legislative",
    },
    "legis2012": {
        "date": "2012-06-10",
        "file": "leg2012.xls",
        "sheet": "Régions T1",
        "denom": "Code Nuance",
        "type": "legislative",
    },
    "legis2017": {
        "date": "2017-06-11",
        "file": "leg2017.xlsx",
        "sheet": "Regions T1",
        "denom": "Code Nuance",
        "type": "legislative",
    },
    "presid2002": {
        "date": "2002-04-21",
        "file": "pres2002.xls",
        "sheet": "Régions T1",
        "denom": "Nom",
        "type": "president",
    },
    "presid2007": {
        "date": "2007-04-22",
        "file": "pres2007.xls",
        "sheet": "Régions T1",
        "denom": "Nom",
        "type": "president",
    },
    "presid2012": {
        "date": "2012-04-22",
        "file": "pres2012.xls",
        "sheet": "Régions T1",
        "denom": "Nom",
        "type": "president",
    },
    "presid2017": {
        "date": "2017-04-23",
        "file": "pres2017.xls",
        "sheet": "Régions Tour 1",
        "denom": "Nom",
        "type": "president",
    },
    "regio2004": {
        "date": "2004-03-21",
        "file": "reg2004.xls",
        "sheet": "Regions T1",
        "denom": "Nuance Liste",
        "type": "regional",
    },
    "regio2010": {
        "date": "2010-03-14",
        "file": "reg2010.xls",
        "sheet": "Régions T1",
        "denom": "Nuance Liste",
        "type": "regional",
    },
    "regio2015": {
        "date": "2015-12-06",
        "file": "reg2015.xlsx",
        "sheet": "Régions",
        "denom": "Nuance Liste",
        "type": "regional",
    },
}

As the formatting is almost the same for all the files, let's write some handy functions. These two functions basically turn the raw file into a format that we can give to the subsequent `format_results` function.

In [11]:
def load_and_clean(election: Dict[str, Dict[str, str]], header=0) -> pd.DataFrame:
    """
    Load file for given election, select only Paris, add election date and label districts (aka arrondissements).
    """
    df = pd.read_excel(
        f"raw_election_results_1st_round/{election['file']}",
        header=header,
        sheet_name=election["sheet"],
    )
    df["election_day"] = pd.to_datetime(election["date"])

    return (
        df.rename(
            columns={
                "Code de la région": "region_code",
                "Libellé de la région": "region",
            }
        )
        .sort_values("region")
        .reset_index(drop=True)
    )


def select_columns(df: pd.DataFrame, party_col: str) -> pd.DataFrame:
    """
    party_col: value of the 'denom' key in the dict of elections.
    """
    subset = ["election_day", "region", "Exprimés"]
    for nuance, score in zip(
        df.filter(like=party_col).columns, df.columns[df.columns.str.startswith("Voix")]
    ):
        subset.append(nuance)
        subset.append(score)

    return df[subset]

Let's detail how we do it for the 2002 presidential election, and then we'll do all the elections in one pass:

In [24]:
d = load_and_clean(ELECTIONS["presid2002"])
d = select_columns(d, ELECTIONS["presid2002"]["denom"])
d.head()

,election_day,region,Exprimés,Nom,Voix,Nom.1,Voix.1,Nom.2,Voix.2,Nom.3,...,Nom.11,Voix.11,Nom.12,Voix.12,Nom.13,Voix.13,Nom.14,Voix.14,Nom.15,Voix.15
0,2002-04-21,ALSACE,821523,MEGRET,35661,LEPAGE,17815,GLUCKSTEIN,3738,BAYROU,...,HUE,8029,CHEVENEMENT,43169,MADELIN,36264,LAGUILLER,40169,BESANCENOT,28443
1,2002-04-21,AQUITAINE,1490862,MEGRET,23272,LEPAGE,23795,GLUCKSTEIN,6042,BAYROU,...,HUE,58342,CHEVENEMENT,64958,MADELIN,41747,LAGUILLER,81335,BESANCENOT,69223
2,2002-04-21,AUVERGNE,692325,MEGRET,13968,LEPAGE,11568,GLUCKSTEIN,3847,BAYROU,...,HUE,31097,CHEVENEMENT,38972,MADELIN,23120,LAGUILLER,46042,BESANCENOT,36260
3,2002-04-21,BASSE NORMANDIE,735412,MEGRET,14293,LEPAGE,15252,GLUCKSTEIN,3933,BAYROU,...,HUE,15565,CHEVENEMENT,33474,MADELIN,28526,LAGUILLER,51022,BESANCENOT,34729
4,2002-04-21,BOURGOGNE,805358,MEGRET,22107,LEPAGE,14062,GLUCKSTEIN,4477,BAYROU,...,HUE,27094,CHEVENEMENT,46053,MADELIN,33138,LAGUILLER,47119,BESANCENOT,36514


Below, the functions `extract_nuances` and `format_results` are designed to extract the unique nuances competing in the given election, and then match each nuance with the corresponding score of the party.

In [13]:
def extract_nuances(nuances_df: pd.DataFrame) -> Set[str]:
    """
    Extract the nuances competing in this election.
    From the dataframe of nuances, we check each column for each line.
    If the cell is not empty and the nuance is not already counted, we add it to the set of nuances.
    """
    nuances_set = set()

    for _, line in nuances_df.iterrows():
        for col in nuances_df.columns:
            if pd.notnull(line[col]):
                nuances_set.update({line[col]})

    return nuances_set


def format_results(df: pd.DataFrame, nuances_set: Set[str]) -> pd.DataFrame:
    """
    Take the raw df, for each line switch the nuance's label to column name,
    and match it with the corresponding score of this party.
    Return a dataframe with the proper format.
    """
    res = {
        "election_day": df["election_day"].values,
        "region": df["region"].values,
        "Exprimés": df["Exprimés"].values,
    }
    res.update({nuance: [] for nuance in nuances_set})

    if not df.filter(like="Code Nuance").columns.empty:
        nuances_lbls = df.filter(like="Code Nuance").columns
    elif not df.filter(like="Nuance").columns.empty:
        nuances_lbls = df.filter(like="Nuance").columns
    elif not df.filter(like="Nuance Liste").columns.empty:
        nuances_lbls = df.filter(like="Nuance Liste").columns
    elif not df.filter(like="Nom").columns.empty:
        nuances_lbls = df.filter(like="Nom").columns
    elif not df.filter(like="Libellé Abrégé Liste").columns.empty:
        nuances_lbls = df.filter(like="Libellé Abrégé Liste").columns
    else:
        raise ValueError("nuances_lbls seems empty.")
    scores_lbls = df.filter(like="Voix").columns

    # each line is a subunit (region, department):
    for _, line in df.iterrows():
        tempset = nuances_set.copy()

        # iterate over nuances in line:
        for n, s in zip(nuances_lbls, scores_lbls):
            name = line[n]
            score = line[s]
            if pd.notnull(name):
                # if 1st time we see this nuance in this line:
                if name in tempset:
                    res[name].append(score)
                    tempset.remove(name)
                # if we already saw this nuance in this line:
                else:
                    res[name][-1] += score
        # if nuance still in tempset after iteration, then it's not competing in this subunit:
        for nuance in tempset:
            res[nuance].append(np.nan)

    return pd.DataFrame(data=res)

Then, we have to attribute each colloquial party name to its general party denomination. However, some parties may have the same ideological leaning, or they have made alliance, so we have to add them together. The function `attribute_parties` takes care of it, and then aggregates the rest of the parties into the category "other". Finally, it drops all useless parties.

In [25]:
def attribute_parties(df: pd.DataFrame, nuances_set: Set[str]) -> pd.DataFrame:
    """
    From a dataframe with general party denomination, attribute colloquial party names and
    add parties with same nuance.
    Then aggregate the rest of the parties, drop all useless ones, and reorder columns.
    """
    for p in AFFILIATIONS.keys():
        # which candidate represents the party this year?
        intersection = list(nuances_set & set(AFFILIATIONS[p]))
        # take only LFI for farleft, starting in 2012:
        if (
            ("LFG" in intersection)
            or ("FG" in intersection)
            or ("MÉLENCHON" in intersection)
            or ("FI" in intersection)
            or ("BC-FG" in intersection)
        ):
            df = df.rename(
                columns={"LFG": p, "FG": p, "MÉLENCHON": p, "FI": p, "BC-FG": p}
            )
        else:
            # add candidates with same nuance, then drop:
            if len(intersection) >= 2:
                df[p] = df[intersection].sum(axis=1)
                df.drop(intersection, axis=1, inplace=True)
            # rename column of only candidate of this party:
            elif len(intersection) == 1:
                df = df.rename(columns={intersection[0]: p})

    # aggregate other parties:
    core_cols = ["election_day", "region", "Exprimés"] + list(AFFILIATIONS.keys())
    rest = df[df.columns.difference(core_cols)]
    df["other"] = rest.sum(axis=1)
    df.drop(rest.columns, axis=1, inplace=True)

    # reorder columns:
    df = df.rename(columns={"Exprimés": "N"})
    df = df.reindex(
        ["election_day", "region", "N"] + list(AFFILIATIONS.keys()) + ["other"],
        axis=1,
    )

    return df

And now we can use all of this:

In [23]:
nuances_set = extract_nuances(d.filter(like=ELECTIONS["presid2002"]["denom"]))
d = format_results(d, nuances_set)
d = attribute_parties(d, nuances_set)
d["type"] = ELECTIONS["presid2002"]["type"]
d

,election_day,region,N,farleft,left,green,center,right,farright,other,type
0,2002-04-21,ALSACE,821523,68612,90494,49411,88529,150818,192584,181075,president
1,2002-04-21,AQUITAINE,1490862,150558,270248,81190,117346,285053,196129,390338,president
2,2002-04-21,AUVERGNE,692325,82302,108557,30770,48439,147066,98713,176478,president
3,2002-04-21,BASSE NORMANDIE,735412,85751,106853,34159,50445,164490,108403,185311,president
4,2002-04-21,BOURGOGNE,805358,83633,130481,35513,52855,153194,147046,202636,president
5,2002-04-21,BRETAGNE,1585745,189416,286515,102128,113920,342538,187323,363905,president
6,2002-04-21,CENTRE,1221727,125029,184174,55098,82415,244240,210385,320386,president
7,2002-04-21,CHAMPAGNE-ARDENNE,627490,67083,88993,26572,41469,125325,132539,145509,president
8,2002-04-21,CORSE,109462,6629,16836,3265,3660,30180,17166,31726,president
9,2002-04-21,FRANCHE-COMTE,572349,58790,77388,27953,34985,101328,114376,157529,president


Got it? Let's do the same thing for all the elections at the same time now:

In [173]:
results = []
for election in ELECTIONS.values():
    print(f"Formatting {election['file']}...\n")
    df = load_and_clean(election)
    df = select_columns(df, election["denom"])

    nuances_set = extract_nuances(df.filter(like=election["denom"]))
    df = format_results(df, nuances_set)
    df = attribute_parties(df, nuances_set)
    df["type"] = election["type"]

    results.append(df)

results = (
    pd.concat(results)
    .sort_values(["election_day", "type", "region"])
    .reset_index(drop=True)
)
results

Formatting dep2004.xls...

Formatting dep2008.xls...

Formatting dep2011.xls...

Formatting euro2004.xls...

Formatting euro2009.xls...

Formatting euro2014.xlsx...

Formatting euro2019-regions.xls...

Formatting leg2002.xls...

Formatting leg2007.xls...

Formatting leg2012.xls...

Formatting leg2017.xlsx...

Formatting pres2002.xls...

Formatting pres2007.xls...

Formatting pres2012.xls...

Formatting pres2017.xls...

Formatting reg2004.xls...

Formatting reg2010.xls...

Formatting reg2015.xlsx...



,election_day,region,N,farleft,left,green,center,right,farright,other,type
0,2002-04-21,ALSACE,821523,68612.0,90494.0,49411.0,88529.0,150818.0,192584.0,181075.0,president
1,2002-04-21,AQUITAINE,1490862,150558.0,270248.0,81190.0,117346.0,285053.0,196129.0,390338.0,president
2,2002-04-21,AUVERGNE,692325,82302.0,108557.0,30770.0,48439.0,147066.0,98713.0,176478.0,president
3,2002-04-21,BASSE NORMANDIE,735412,85751.0,106853.0,34159.0,50445.0,164490.0,108403.0,185311.0,president
4,2002-04-21,BOURGOGNE,805358,83633.0,130481.0,35513.0,52855.0,153194.0,147046.0,202636.0,president
...,...,...,...,...,...,...,...,...,...,...,...
433,2019-05-26,Nouvelle-Aquitaine,2222990,156342.0,172774.0,295549.0,495592.0,166505.0,496762.0,439466.0,european
434,2019-05-26,Occitanie,2186389,162093.0,165841.0,288703.0,439620.0,153909.0,562721.0,413502.0,european
435,2019-05-26,Pays de la Loire,1342007,71373.0,93308.0,209027.0,337679.0,117138.0,252548.0,260934.0,european
436,2019-05-26,Provence-Alpes-Côte d'Azur,1710441,100001.0,76667.0,200341.0,348437.0,150738.0,521969.0,312288.0,european


It all worked well, but we're not done yet, I have a special treat for you. In 2013, some regions were merged, giving birth to fewer, bigger regions, so we have to account for that and aggregate our results by new regions for elections prior to 2013. 

We need to deal with strings to do that, which is always harder. After some trial and error, I came up with the following code, which does what we're after: associating old region names to the new names, and then aggregating the results.

In [174]:
# remove accents and special characters, then remove hyphens
results["region"] = (
    results.region.str.lower()
    .str.normalize("NFKD")
    .str.encode("ascii", errors="ignore")
    .str.decode("utf-8")
    .str.replace("-", " ")
)

In [175]:
new_old_regions = pd.read_csv(
    "raw_election_results_1st_round/anciennes-nouvelles-regions.csv", sep=";"
).sort_values("Anciens Nom")
old_region_matching = (
    new_old_regions.groupby("Ancien nom normalisé").first()["Nouveau Nom"].to_dict()
)
new_region_matching = (
    new_old_regions.groupby("Nouveau nom normalisé").first()["Nouveau Nom"].to_dict()
)

In [176]:
results = (
    results.replace(
        {"region": {"pays de loire": "pays de la loire"}}
    )  # due to a typo somewhere in a raw file
    .replace({"region": old_region_matching})
    .replace(
        {"region": new_region_matching}
    )  # because some regions already have right names
    .sort_values(["election_day", "type", "region"])
    .reset_index(drop=True)
)

In [177]:
# if True, we did our job
assert np.array_equal(
    new_old_regions["Nouveau Nom"].sort_values().unique(),
    results.region.sort_values().unique(),
)

In [178]:
# sum results by new regions and save to disk
results = results.groupby(["election_day", "type", "region"]).sum()
results.to_csv("results_by_regions.csv")
results

N   farleft  \
election_day type      region                                          
2002-04-21   president Auvergne-Rhône-Alpes        3321976  323854.0   
                       Bourgogne-Franche-Comté     1377707  142423.0   
                       Bretagne                    1585745  189416.0   
                       Centre-Val de Loire         1221727  125029.0   
                       Corse                        109462    6629.0   
...                                                    ...       ...   
2019-05-26   european  Normandie                   1211420   75665.0   
                       Nouvelle-Aquitaine          2222990  156342.0   
                       Occitanie                   2186389  162093.0   
                       Pays de la Loire            1342007   71373.0   
                       Provence-Alpes-Côte d'Azur  1710441  100001.0   

                                                       left     green  \
election_day type      region                                           
2002-04-21   president Auvergne-Rhône-Alpes        472029.0  191041.0   
                       Bourgogne-Franche-Comté     207869.0   63466.0   
                       Bretagne                    286515.0  102128.0   
                       Centre-Val de Loire         184174.0   55098.0   
                       Corse                        16836.0    3265.0   
...                                                     ...       ...   
2019-05-26   european  Normandie                    71759.0  137325.0   
                       Nouvelle-Aquitaine          172774.0  295549.0   
                       Occitanie                   165841.0  288703.0   
                       Pays de la Loire             93308.0  209027.0   
                       Provence-Alpes-Côte d'Azur   76667.0  200341.0   

                                                     center     right  \
election_day type      region                                           
2002-04-21   president Auvergne-Rhône-Alpes        253051.0  597258.0   
                       Bourgogne-Franche-Comté      87840.0  254522.0   
                       Bretagne                    113920.0  342538.0   
                       Centre-Val de Loire          82415.0  244240.0   
                       Corse                         3660.0   30180.0   
...                                                     ...       ...   
2019-05-26   european  Normandie                   253703.0   95280.0   
                       Nouvelle-Aquitaine          495592.0  166505.0   
                       Occitanie                   439620.0  153909.0   
                       Pays de la Loire            337679.0  117138.0   
                       Provence-Alpes-Côte d'Azur  348437.0  150738.0   

                                                   farright     other  
election_day type      region                                          
2002-04-21   president Auvergne-Rhône-Alpes        620297.0  864446.0  
                       Bourgogne-Franche-Comté     261422.0  360165.0  
                       Bretagne                    187323.0  363905.0  
                       Centre-Val de Loire         210385.0  320386.0  
                       Corse                        17166.0   31726.0  
...                                                     ...       ...  
2019-05-26   european  Normandie                   322381.0  255307.0  
                       Nouvelle-Aquitaine          496762.0  439466.0  
                       Occitanie                   562721.0  413502.0  
                       Pays de la Loire            252548.0  260934.0  
                       Provence-Alpes-Côte d'Azur  521969.0  312288.0  

[312 rows x 8 columns]

In [179]:
%load_ext watermark
%watermark -a AlexAndorra -n -u -v -iv

Author: AlexAndorra

Last updated: Mon May 31 2021

Python implementation: CPython
Python version       : 3.9.2
IPython version      : 7.22.0

numpy : 1.20.2
pandas: 1.2.4

